In [212]:
import pandas as pd
import os
import numpy as np
import string
import re

os.getcwd()
path = 'C:\\Users\\Mateusz Jodlowski\\Desktop\\mgr'
os.chdir(path)

movies = pd.read_csv('t-sne\\tmdb-movies.csv')
movies = movies.rename(columns = {'genres' : 'Genre 1', 'overview' : 'Description'})
movies = movies[((~movies['Genre 1'].str.split('|',expand=True).isnull()).apply(sum, axis = 1) == 1).values].reset_index()
movies = movies[movies.columns.difference(['index'])]
movies = movies[['Genre 1', 'Description']]
movies = movies[(movies.isnull().apply(sum, axis = 1) == 0).values]
movies

,Genre 1,Description
0,Action,Ethan and team take on their most impossible m...
1,Comedy,Newlywed couple Ted and Tami-Lynn want to have...
2,Comedy,70-year-old widower Ben Whittaker has discover...
3,Drama,Adam Jones is a Chef who destroyed his career ...
4,Drama,When Gerda Wegener asks her husband Einar to f...
5,Drama,The former World Heavyweight Champion Rocky Ba...
6,Comedy,Having thought that monogamy was never possibl...
7,Thriller,A high-rolling corporate shark and his impover...
8,Drama,A dramatic thriller based on the incredible tr...
9,Comedy,In New York City for their annual tradition of...


In [228]:
movies.reset_index()

,index,Genre 1,Description
0,0,Action,Ethan and team take on their most impossible m...
1,1,Comedy,Newlywed couple Ted and Tami-Lynn want to have...
2,2,Comedy,70-year-old widower Ben Whittaker has discover...
3,3,Drama,Adam Jones is a Chef who destroyed his career ...
4,4,Drama,When Gerda Wegener asks her husband Einar to f...
5,5,Drama,The former World Heavyweight Champion Rocky Ba...
6,6,Comedy,Having thought that monogamy was never possibl...
7,7,Thriller,A high-rolling corporate shark and his impover...
8,8,Drama,A dramatic thriller based on the incredible tr...
9,9,Comedy,In New York City for their annual tradition of...


In [227]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.lancaster import LancasterStemmer
stop_words = set(stopwords.words('english'))

docs = movies['Description']
docs = docs.apply(lambda x: x.translate(str.maketrans({key: None for key in string.punctuation})))
docs = docs.apply(lambda x: x.lower())

word_tokens = docs.apply(word_tokenize)
docs = word_tokens.apply(lambda x: [w for w in x if not w in stop_words])

wordnet_lemmatizer = WordNetLemmatizer()
lancaster_stemmer = LancasterStemmer()

docs = docs.apply(lambda x: [wordnet_lemmatizer.lemmatize(w) for w in x])
docs = docs.apply(lambda x: [lancaster_stemmer.stem(w) for w in x])
docs = docs.apply(lambda x: ' '.join(x))

vec = CountVectorizer()

X = vec.fit_transform(docs)

pd.DataFrame(X.toarray())



# dtm = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

# dtm = dtm.iloc[:, pd.Series(dtm.columns.values).apply(lambda x: re.match("^[0-9]", x)).isnull().values]
# dtm = dtm.iloc[:, pd.Series(dtm.columns.values).apply(lambda x: re.match("^œ", x)).isnull().values]
# dtm

,0,1,2,3,4,5,6,7,8,9,...,10871,10872,10873,10874,10875,10876,10877,10878,10879,10880
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [224]:
dtm = dtm.iloc[:, ((dtm > 0).sum() > 1).values]

In [215]:
class_input = pd.concat([movies['Genre 1'], dtm], axis = 1)

In [216]:
class_input

,Genre 1,aaron,ab,abandon,abby,abc,abduc,abl,aboard,abort,...,zac,zach,zack,zan,zealand,zebr,zero,zism,zomby,zon
0,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
class_input

,Genre 1,aaron,ab,abandon,abby,abc,abduc,abl,aboard,abort,...,zac,zach,zack,zan,zealand,zebr,zero,zism,zomby,zon
0,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
feed = first_class[first_class.columns.difference(['Genre 1'])]

In [99]:
from sklearn.manifold import TSNE

In [109]:
first_class_embedded = TSNE(n_components=2).fit_transform(feed)

In [128]:
first_class_embedded = pd.DataFrame(first_class_embedded)

In [129]:
first_class_embedded['target'] = dtm['Genre 1']

In [130]:
first_class_embedded

,0,1,target
0,-0.335434,0.177084,Action
1,5.548445,1.539074,Adventure
2,0.578390,-0.466535,Horror
3,5.980873,-4.785925,Animation
4,-0.305887,3.576632,Action
5,0.158712,1.574919,Action
6,2.866910,-1.870351,Comedy
7,-4.781955,-0.294170,Comedy
8,1.312897,3.943278,Action
9,5.019880,-2.370740,Adventure


In [154]:
first_class_embedded.columns = ['x', 'y', 'target']

In [151]:
import matplotlib.pyplot as plt
import seaborn
seaborn.set(style='ticks')
import pylab

In [174]:
fg = seaborn.FacetGrid(data=first_class_embedded, hue='target')

In [175]:
fig = fg.map(plt.scatter, 'x', 'y').add_legend()

In [176]:
pylab.savefig('first_view.png')